# Agent Engine の A2A 対応機能の利用例 (1)

Agent Engine に A2A サーバーの機能がデプロイできるようになりました。

たとえば、下記の記事では、Agent Engine にデプロイした ADK Agent の前段に Cloud Run を用いて A2A サーバーを用意していましたが、Agent Engine に A2A サーバーの機能も同時にデプロイできるようになります。

- [A2A リモートエージェント対応のマルチエージェントシステム](https://zenn.dev/google_cloud_jp/articles/d24a22a87a84f8)

このノートブックでは、上記の記事で Cloud Run にデプロイしていた A2A サーバーの機能をそのままの形で Agent Engine に移行する例を紹介します。

※ この例では、A2A サーバー、および、A2A クライアントの機能は独自に実装していますが、ADK および Agent Engine が提供する A2A 対応モジュールを利用すると、より簡単に実装することもできます。

## パッケージインストール

In [ ]:
%pip install --user \
    google-adk==1.13.0 \
    google-genai==1.36.0 \
    google-cloud-aiplatform==1.112.0 \
    a2a-sdk==0.3.5

カーネルを再起動します。

In [3]:
import IPython
app = IPython.Application.instance()
_ = app.kernel.do_shutdown(True)

## 事前準備

In [1]:
import copy, json, os, uuid
from datetime import datetime, timezone
import vertexai
from vertexai import agent_engines

from google.genai.types import Part, Content, HttpOptions
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.events import Event
from google.adk.runners import Runner
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.sequential_agent import SequentialAgent

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
[PROJECT_NUMBER] = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

root agent をローカルで実行する簡易アプリのクラスを定義します。

In [2]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'====\n{event}\n====')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

## リモートエージェントの定義

In [3]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [4]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
前段のエージェントは、５項目程度の調査対象トピックを指定します。

* 出力形式
日本語で出力。
調査レポートは、トピックごとに客観的情報をまとめます。各トピックについて、５文以上の長さで記述すること。
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    instruction=instruction,
)

In [5]:
instruction = '''
あなたの役割は、特定のテーマに関する気軽な読み物記事を書くことです。
記事の「テーマ」と、その内容に関連する「調査レポート」が与えられるので、
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

レビュアーによる修正ポイントが指摘された際は、直前に書いた記事を指摘に従って書き直してください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 記事タイトルは付けないで、次の構成で出力します。各セクションタイトルは、内容に合わせてアレンジしてください。
0. 導入：セクションタイトルを付けないで、この記事を読みたくなる導入を１〜２文にまとめます。
1. 概要：トピックの全体像をまとめて簡単に説明します。
2. 最新情報：特に注目したい新しい情報を取り上げます。
3. 実践：トピックに関して、読者自身がやってみるとよさそうな事を１つ紹介します。
4. まとめ

- 各セクションのタイトルはマークダウンヘッダ ## 付けます。必要に応じて箇条書きのマークダウンを使用します。
- それ以外のマークダウンによる装飾は行いません。
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [6]:
instruction = f'''
あなたの役割は、読み物記事をレビューして、記事の条件にあった内容にするための改善コメントを与える事です。

* 記事の条件
- 記事は、はじめに４０文字程度のタイトルがあること。
　今日から役立つ生活情報があって「すぐに読まなきゃ」と読者が感じるタイトルにすること。
　タイトルはマークダウンヘッダ # をつけること。
- タイトルの直後に「なぜいまこのテーマを取り上げるのか」をまとめた導入を加えて、読者にこの記事を読む動機づけを与えます。
- 各セクションのサブタイトルには、絵文字を用いて親しみやすさを出すこと。
- 読者が今日から実践できる具体例が３つ以上紹介されていること。

* 出力形式
- 日本語で出力。
- はじめに、記事の良い点を説明します。
- 次に、修正ポイントを箇条書きで出力します。
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    instruction=instruction,
)

## Agent Engine にデプロイ

In [7]:
from a2a.client import ClientConfig, ClientFactory
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import TaskUpdater
from a2a.types import (
    AgentSkill,
    Message,
    TaskArtifactUpdateEvent,
    TaskState,
    TextPart,
    UnsupportedOperationError,
)
from a2a.utils import new_agent_text_message
from a2a.utils.errors import ServerError

from vertexai.preview.reasoning_engines import A2aAgent
from vertexai.preview.reasoning_engines.templates.a2a import create_agent_card

Agent card を生成する補助関数

In [8]:
def get_agent_card(agent):
    agent_name = agent.name
    agent_description = agent.description
    skill = AgentSkill(
        id=f"{agent_name.lower().replace(' ', '_')}_skill",
        name=f'{agent_name}',
        description=agent_description,
        tags=[tag for tag in agent_name.split('_')[:-1]],
        examples=[f'Interact with {agent_name}'],
    )
    return create_agent_card(
        agent_name=agent_name,
        description=agent_description,
        skills=[skill],
    )

A2A サーバーモジュール（AgentExecutor）を生成する補助関数

In [9]:
class AdkAgent:
    def __init__(self, agent):
        self.agent = agent
        self.runner = Runner(
            app_name=self.agent.name,
            agent=self.agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )

    async def stream(self, updater, user_input):
        session_service = self.runner.session_service
        session = await session_service.create_session(
            app_name=self.runner.app_name,
            user_id='default_user',
        )
        app_name = session.app_name
        user_id = session.user_id
        session_id = session.id

        for event in json.loads(user_input): # past events
            await session_service.append_event(session, Event(**event))
  
        events = self.runner.run_async(
            user_id=user_id,
            session_id=session_id,
            # Sequential agents receive no user messages.
            new_message=Content(role='user', parts=[Part(text='')]),
        )
        
        text_parts = []        
        async for event in events:
            if text_parts: # not a last part
                updater.new_agent_message(text_parts)
            text_parts = []
            if (event.content and event.content.parts):
                for part in event.content.parts:
                    if part.text:
                        text_parts.append(TextPart(text=part.text))
        # treat last part as an artifact
        await updater.add_artifact(text_parts, name='responseText')
        await session_service.delete_session(
            app_name=app_name,
            user_id=user_id,
            session_id=session_id,
        )


class _AdkAgentExecutor(AgentExecutor):
    def __init__(self):
        self.adk_agent = None

    def _init_agent(self):
        pass

    async def execute(self, context, event_queue):
        if self.adk_agent is None:
            self._init_agent()

        updater = TaskUpdater(
            event_queue,
            context.task_id,
            context.context_id
        )
        if not context.current_task:
            await updater.submit()        
        await updater.start_work()
        user_input = context.get_user_input()
        try:
            await self.adk_agent.stream(updater, user_input)
            await updater.complete()
        except Exception as e:
            await updater.update_status(
                TaskState.failed, message=new_agent_text_message(f'Error: {e!s}')
            )
            raise

    async def cancel(self, context, event_queue):
        raise ServerError(error=UnsupportedOperationError())
        

def get_agent_executor_class(agent):
    class AdkAgentExecutor(_AdkAgentExecutor):
        def __init__(self):
            super().__init__()

        def _init_agent(self):
            self.adk_agent = AdkAgent(agent)
            
    return AdkAgentExecutor

エージェント毎に Agent card を A2A サーバーモジュールを生成して、Agent Engine にデプロイ

In [10]:
agents = {
    'research_agent1_a2a': research_agent1,
    'research_agent2_a2a': research_agent2,
    'writer_agent_a2a': writer_agent,
    'review_agent_a2a': review_agent,
}

deployed_agents = [agent.display_name for agent in agent_engines.list()]

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version="v1beta1", base_url=f"https://{LOCATION}-aiplatform.googleapis.com/"
    ),
)

for agent_name, agent in agents.items():
    a2a_agent = A2aAgent(
        agent_card=get_agent_card(agent),
        agent_executor_builder=get_agent_executor_class(agent),
    )
    config={
        "display_name": agent_name,
        "description": a2a_agent.agent_card.description,
        "requirements": [
            'google-adk==1.14.1',
            'google-genai==1.36.0',
            'google-cloud-aiplatform==1.113.0',
            'a2a-sdk==0.3.5'
        ],
        "http_options": {
            "base_url": f"https://{LOCATION}-aiplatform.googleapis.com",
            "api_version": "v1beta1",
        },
        "staging_bucket": f'gs://{PROJECT_ID}',
    }

    if not agent_name in deployed_agents:
        remote_a2a_agent = client.agent_engines.create(
            agent=a2a_agent,
            config=config,
        )
    else:
        resource_name = [
            agent.resource_name for agent in agent_engines.list()
            if agent.display_name == agent_name
        ][0]
        remote_a2a_agent = client.agent_engines.update(
            name=resource_name,
            agent=a2a_agent,
            config=config,
        )

The following requirements are missing: {'cloudpickle', 'a2a', 'pydantic'}
The following requirements are missing: {'cloudpickle', 'a2a', 'pydantic'}
The following requirements are missing: {'cloudpickle', 'a2a', 'pydantic'}
The following requirements are missing: {'cloudpickle', 'a2a', 'pydantic'}


## プロキシーエージェントの定義

コールバック関数内で A2A クライアントを用いてリモートエージェントを実行する「プロキシーエージェント」を定義します。

In [12]:
from google.auth import default
from google.auth.transport.requests import Request
from a2a.client import ClientConfig, ClientFactory
from a2a.types import TransportProtocol, TaskQueryParams
import httpx

def get_call_a2a_agent(agent_name):
    remote_a2a_agent = None
    client = vertexai.Client(
        project=PROJECT_ID,
        location=LOCATION,
        http_options=HttpOptions(
            api_version='v1beta1', base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
        )
    )
    for agent in agent_engines.list():
        if agent.display_name == agent_name:
            config = {'http_options': {'base_url': f'https://{LOCATION}-aiplatform.googleapis.com'}}
            remote_a2a_agent = client.agent_engines.get(
                name=agent.resource_name,
                config=config,
            )
            break

    async def call_a2a_agent(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> Content:
        
        # Prepare A2A client.
        credentials, _ = default(scopes=['https://www.googleapis.com/auth/cloud-platform'])
        credentials.refresh(Request())
        factory = ClientFactory(
            ClientConfig(
                supported_transports=[TransportProtocol.http_json],
                use_client_preference=True,
                httpx_client=httpx.AsyncClient(
                    timeout=60, # This is important.
                    headers={
                        'Authorization': f'Bearer {credentials.token}',
                        'Content-Type': 'application/json',
                    }
                ),
            )
        )
        remote_a2a_agent_card = await remote_a2a_agent.handle_authenticated_agent_card()
        a2a_client = factory.create(remote_a2a_agent_card)
        
        # Convert past events into JSON serializable format.
        events = callback_context._invocation_context.session.events
        events = [event.model_dump() for event in events]
        for event in events:
            for key, item in event.items():
                if isinstance(item, set):
                    event[key] = list(item)
                    
        message_data = {
            'messageId': str(uuid.uuid4()),
            'role': 'user',
            'parts': [{'kind': 'text', 'text': json.dumps(events)}],
        }
        response = a2a_client.send_message(message_data)

        artifacts = None
        async for chunk in response:
            if isinstance(chunk, tuple) and len(chunk) > 0 and hasattr(chunk[0], 'artifacts'):
                artifacts = chunk[0].artifacts
                break
        result = []
        for artifact in artifacts:
            text_messages = [part.root.text for part in artifact.parts]
            result.append(Part(text='\n'.join(text_messages)))
        return LlmResponse(
            content=Content(role='model', parts=result) 
        )

    return call_a2a_agent

In [13]:
research_agent1_proxy = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash', # not used
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    before_model_callback=get_call_a2a_agent('research_agent1_a2a'),
)

research_agent2_proxy = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash', # not used
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    before_model_callback=get_call_a2a_agent('research_agent2_a2a'),
)

writer_agent_proxy = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash', # not used
    description='特定のテーマに関する読み物記事を書くエージェント',
    before_model_callback=get_call_a2a_agent('writer_agent_a2a'),
)

review_agent_proxy = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash', # not used
    description='読み物記事をレビューするエージェント',
    before_model_callback=get_call_a2a_agent('review_agent_a2a'),
)

## Root agent の定義

プロキシーエージェントを取りまとめてサブエージェントにした root agent を定義します。

In [14]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
        return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        copy.deepcopy(research_agent1_proxy),
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        copy.deepcopy(research_agent2_proxy),
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        copy.deepcopy(writer_agent_proxy),
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        copy.deepcopy(review_agent_proxy),
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

In [15]:
root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        copy.deepcopy(research_agent),
        copy.deepcopy(write_and_review_agent),
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

## 実行例

In [27]:
client = LocalApp(root_agent)
DEBUG = False

query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！私は、ユーザーさんが指定したテーマで記事を作成するお手伝いをします。

まず、リサーチエージェントがテーマに関する調査レポートを作成します。

その後、ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。

記事のテーマを指定していただければ、すぐに作成に取り掛かります！



In [28]:
query = '''
「夏の節電対策」をテーマに記事を作成してください。
'''
result = await client.stream(query)

はい、「夏の節電対策」の記事作成に取り掛かります。

まず、リサーチエージェントに調査レポートを依頼します。その後、記事の作成とレビューのためにライターエージェントとレビューエージェントに転送します。


---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。

「夏の節電対策」の記事作成に向けた調査レポートのためのトピック案を以下にまとめます。このトピックに基づいて、詳細な情報を収集します。

---

**【調査レポート トピックリスト：夏の節電対策】**

1.  **夏の節電が求められる背景とメリット**:
    *   なぜ夏に電力消費が増えるのか（エアコン使用など）。
    *   電力需給の逼迫、電気代高騰、地球温暖化といった背景。
    *   節電によって得られる具体的なメリット（電気代削減、CO2排出量削減、環境負荷低減など）。
2.  **エアコンを賢く使う節電術**:
    *   設定温度の目安と快適さを保つ工夫（扇風機・サーキュレーターの併用）。
    *   定期的なフィルター掃除の重要性。
    *   室外機の効率を高める対策（日よけ、周囲の片付け）。
    *   自動運転モードの活用。
3.  **家庭で実践できるその他の主要な節電対策**:
    *   冷蔵庫：設定温度の見直し、開閉頻度・時間の短縮、詰め込みすぎ防止。
    *   照明：LED照明への交換、こまめな消灯、自然光の活用。
    *   テレビ・パソコン：画面輝度調整、省エネモードの活用、こまめな電源オフ。
    *   待機電力の削減方法（電源タップの利用など）。
4.  **節電をサポートする便利グッズ・サービスと住まいの工夫**:
    *   遮熱・断熱カーテン、窓用フィルム、打ち水などの活用。
    *   スマート家電や電力消費が見える化できるサービスの紹介。
    *   電力会社の料金プラン見直しによる節電効果。
    *   長期的な視点での住まいの断熱性向上。
5.  **節電と健康・快適性の両立の注意点**:
    *   熱中症予防の重要性と無理な節電の危険性。
    *   高齢者や乳幼児がいる家庭での配慮事項。
    *   体調管理と快

In [29]:
query = '''
お願いします。
'''
result = await client.stream(query)

承知いたしました。それでは、ライターエージェントとレビューエージェントに記事の作成を依頼します。


---
## ライターエージェントが記事を執筆します。
---

今年も暑い夏がやってきましたね。電気代の高騰や地球温暖化が気になる中、夏の節電は私たちの暮らしにとってますます重要になっています。

## 夏の節電ってなぜ必要？その大きなメリットとは

毎年夏になると、私たちの電力消費量はぐっと増えます。その大きな理由はもちろんエアコン！猛暑日が続く日本では、エアコンはもはや生活必需品ですよね。でも、その分電気代はかさむし、発電のために排出されるCO2も気になるところ。さらに、電力需要が高まりすぎると、地域によっては「電力需給逼迫」なんて言葉も耳にするようになります。

でも、節電は単に電気代を安くするだけじゃないんです。
まず、家計に優しいのは言うまでもありませんが、電力消費を抑えることは、地球温暖化の原因となるCO2の排出量を減らすことにも繋がります。つまり、私たちの未来の環境を守る一歩になるということ。さらに、電力需要のピークを抑えることで、大規模な停電のリスクを減らし、安定した電力供給にも貢献できるんです。

## 最新技術で快適に！「見える化」と「最適化」の時代

「節電」と聞くと、昔はひたすら「我慢」のイメージだったかもしれません。でも、今はテクノロジーの力で、もっと賢く快適に節電できる時代になっています。

例えば、**スマート家電**。スマートフォンと連携して外出先からエアコンを操作したり、電力使用量をリアルタイムで「見える化」できるサービスも増えています。これらを使えば、どこでどれだけの電力が使われているかが一目瞭然！無駄な消費に気づきやすくなり、より効率的な節電に繋がります。

また、意外と見落としがちなのが、ご家庭の**電力会社の料金プラン**です。ライフスタイルに合わせて最適なプランを選ぶだけで、電気代を大きく削減できる可能性があります。例えば、夜間の電気代が安いプランを選んで、夜に洗濯を回すなどの工夫もできますよね。

## 今日からできる！エアコンの「賢い」使い方

夏の節電の主役は何と言ってもエアコン！少しの工夫で大きな節電効果が期待できます。

### 1. 設定温度は28℃を目安に、扇風機と併用！
環境省も推奨しているように、エアコ

In [30]:
query = '''
お願いします。
'''
result = await client.stream(query)

承知いたしました。レビューエージェントからのフィードバックに基づき、ライターエージェントに記事の修正を依頼します。


---
## ライターエージェントが記事を執筆します。
---

# 電気代爆上がりはもうイヤ！今日からできる夏の節電術【猛暑を乗り切る快適ワザ】

今年の夏も記録的な暑さが予想され、電気代の高騰は家計に大きな負担となっています。でもご安心ください！この記事を読めば、暑さを我慢することなく、賢く電気代を節約し、快適に夏を過ごすための具体的な方法がすぐに分かります。知っているか知らないかで大きく差が出る節電術を、今日から実践してみませんか？

## 夏の節電ってなぜ必要？その大きなメリットとは💡

毎年夏になると、私たちの電力消費量はぐっと増えます。その大きな理由はもちろんエアコン！猛暑日が続く日本では、エアコンはもはや生活必需品ですよね。でも、その分電気代はかさむし、発電のために排出されるCO2も気になるところ。さらに、電力需要が高まりすぎると、地域によっては「電力需給逼迫」なんて言葉も耳にするようになります。

でも、節電は単に電気代を安くするだけじゃないんです。
まず、家計に優しいのは言うまでもありませんが、電力消費を抑えることは、地球温暖化の原因となるCO2の排出量を減らすことにも繋がります。つまり、私たちの未来の環境を守る一歩になるということ。さらに、電力需要のピークを抑えることで、大規模な停電のリスクを減らし、安定した電力供給にも貢献できるんです。

## 最新技術で快適に！「見える化」と「最適化」の時代📱✨

「節電」と聞くと、昔はひたすら「我慢」のイメージだったかもしれません。でも、今はテクノロジーの力で、もっと賢く快適に節電できる時代になっています。

例えば、**スマート家電**。スマートフォンと連携して外出先からエアコンを操作したり、電力使用量をリアルタイムで「見える化」できるサービスも増えています。これらを使えば、どこでどれだけの電力が使われているかが一目瞭然！無駄な消費に気づきやすくなり、より効率的な節電に繋がります。

また、意外と見落としがちなのが、ご家庭の**電力会社の料金プラン**です。ライフスタイルに合わせて最適なプランを選ぶだけで、電気代を大きく削減できる可能性があります。例えば、夜間の電気代が安いプランを選んで

In [31]:
query = '''
これで完成です。完成した記事を表示してください。
'''
result = await client.stream(query)

# 電気代爆上がりはもうイヤ！今日からできる夏の節電術【猛暑を乗り切る快適ワザ】

今年の夏も記録的な暑さが予想され、電気代の高騰は家計に大きな負担となっています。でもご安心ください！この記事を読めば、暑さを我慢することなく、賢く電気代を節約し、快適に夏を過ごすための具体的な方法がすぐに分かります。知っているか知らないかで大きく差が出る節電術を、今日から実践してみませんか？

## 夏の節電ってなぜ必要？その大きなメリットとは💡

毎年夏になると、私たちの電力消費量はぐっと増えます。その大きな理由はもちろんエアコン！猛暑日が続く日本では、エアコンはもはや生活必需品ですよね。でも、その分電気代はかさむし、発電のために排出されるCO2も気になるところ。さらに、電力需要が高まりすぎると、地域によっては「電力需給逼迫」なんて言葉も耳にするようになります。

でも、節電は単に電気代を安くするだけじゃないんです。
まず、家計に優しいのは言うまでもありませんが、電力消費を抑えることは、地球温暖化の原因となるCO2の排出量を減らすことにも繋がります。つまり、私たちの未来の環境を守る一歩になるということ。さらに、電力需要のピークを抑えることで、大規模な停電のリスクを減らし、安定した電力供給にも貢献できるんです。

## 最新技術で快適に！「見える化」と「最適化」の時代📱✨

「節電」と聞くと、昔はひたすら「我慢」のイメージだったかもしれません。でも、今はテクノロジーの力で、もっと賢く快適に節電できる時代になっています。

例えば、**スマート家電**。スマートフォンと連携して外出先からエアコンを操作したり、電力使用量をリアルタイムで「見える化」できるサービスも増えています。これらを使えば、どこでどれだけの電力が使われているかが一目瞭然！無駄な消費に気づきやすくなり、より効率的な節電に繋がります。

また、意外と見落としがちなのが、ご家庭の**電力会社の料金プラン**です。ライフスタイルに合わせて最適なプランを選ぶだけで、電気代を大きく削減できる可能性があります。例えば、夜間の電気代が安いプランを選んで、夜に洗濯を回すなどの工夫もできますよね。

## 今日からできる！エアコンの「賢い」使い方🌬️

夏の節電の主役は何と言ってもエアコン！少しの工夫で大きな節電効果が期待できます。

#